# Goal:

For the entries in the "job metrics" dataframe which have rinfo strings,
extract all the information from those rinfo strings and add that information to the 
dataframe in the form of new columns. 

This includes parsing data from the rinfo strings and converting to the appropriate format.

Then decide which columns to include in the training dataframe that will be passed to the
ML model. After rearranging and renaming those columns, save to a .pkl file.

# Imports and display settings

In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 75


# Selecting features to extract from rinfo strings

The list of all features we will extract from the rinfo strings is contained in rinfo-features.txt.

In [2]:
feature_names_file = open("rinfo-features.txt")
all_rinfo_features = feature_names_file.read().split('\n')

print(all_rinfo_features)


['dates', 'dsnum', 'elon', 'enddate', 'format', 'grid_definition', 'gui', 'lats', 'level', 'lons', 'nlat', 'parameters', 'product', 'slat', 'startdate', 'station', 'tindex', 'wlon']


Some features are entered with slightly different syntax in different rinfo strings.

For example : grid definition may appear as either "grid_definition" or "grid-definition".

The next cell solves this problem by making a dictionary containing lists of alternate
names for each feature.

In [3]:
special_features = ['grid_definition', 'params', 'format']
normal_features = [feat for feat in all_rinfo_features
                   if feat not in special_features]

special_alt_names = {'grid_definition': ['grid_definition', 'grid-definition'],
                     'parameters': ['parameters', 'params', 'parms'],
                     'format': ['format', 'ofmt', 'fmt'],
                     'tindex': ['tindex', 'gindex']}

alternate_names = {feat: [feat] for feat in normal_features}
alternate_names.update(special_alt_names)


# Functions that parse rinfo strings

In [4]:
def handle_missing_rinfo_val(feature, val):
    """Decides the appropriate 'null value' for when a feature is
    not present in an rinfo string.
    """

    if feature == 'gui':
        return False
    elif feature in ['startdate', 'enddate']:
        return pd.NaT
    else:
        return float('nan')


In [5]:
def format_rinfo_val(rinfo, feature, val):
    """Formats a value found in an rinfo string so that it can be entered
    correctly into the pandas dataframe.
    
    Example input: 
    rinfo = '...;elon=76.4;...'
    feature = 'elon'
    val = '76.4'

    Example output: 76.4
    """

    if val == '':
        return handle_missing_rinfo_val(feature, val)

    try:
        if feature in ['elon', 'wlon', 'nlat', 'slat']:
            return float(val)
        elif feature in ['startdate', 'enddate']:
            return pd.to_datetime(val, errors='coerce')
        elif feature in ['gindex', 'tindex']:
            return float(val)
        elif feature == 'gui':
            return True if val == 'yes' else False
        else:
            return val
    except:
        failed_parse.write(f"Could not parse {feature} from {val}. \n")
        failed_parse.write(f"rinfo string: {rinfo} \n")
        return val


In [6]:
def get_val_from_rinfo(rinfo, feature):
    """Finds the value of the given feature in the given rinfo string.
    If the feature is not present in rinfo, calls handle_missing_rinfo_val.
    If the feature is present, uses format_rinfo_val to convert the value 
    to the appropriate data type before returning.
    
    Example input: 
    rinfo = '...;elon=76.4;...' 
    feature = 'elon'

    Example output: 76.4
    """
    
    rinfo = rinfo.replace('%3D', '=')
    
    if ';' in rinfo:
        sep = ';'
    else:
        sep = '&'

    val = ''
    for alternate_name in alternate_names[feature]:
        if rinfo.lower().find(alternate_name) != -1:
            start_ind = rinfo.lower().find(alternate_name) + len(alternate_name) + 1
            end_ind = rinfo.find(sep, start_ind)
            if end_ind != -1:
                val = rinfo[start_ind:end_ind]
            else:
                val = rinfo[start_ind:]
                
    val = format_rinfo_val(rinfo, feature, val)
    return val

# Parsing rinfo strings

In [7]:
def valid_rinfo(rinfo):
    """Decides whether an rinfo string is 'valid.'
    In practice, just serves to filter out a few problematic rinfo strings
    (14 out of the original 59803, or 0.023%).
    """
    if '\\n' in rinfo:
        return False
    elif '76,78,81,83,85,88,90,92,94,96grid_definition' in rinfo:
        return False
    elif 'startDate' in rinfo:
        return False
    else:
        return True


First read in the dataframe and filter out those which have missing or invalid rinfo strings.

In [8]:
df = pd.read_pickle('/glade/work/jdubeau/job-metrics-data.pkl')
df = df[df.rinfo.notnull()]
df['valid_rinfo'] = df.apply(lambda row: valid_rinfo(row['rinfo']), axis = 1)
df = df[df.valid_rinfo]

In [9]:
list(df.columns)

['start_date',
 'end_date',
 'status',
 'req_mem',
 'used_mem',
 'mem_delta',
 'percent_mem',
 'dsid',
 'request_type',
 'request_id',
 'partition_id',
 'rinfo',
 'cpus',
 'valid_rinfo']

Now add all the rinfo features as new columns. In case of any parsing errors, we
keep track of the rinfo strings that failed to parse in a file called failed-parse.txt.

In [10]:
failed_parse = open('failed-parse.txt', 'w')

for feature in all_rinfo_features:
    df[feature] = df.apply(lambda row: get_val_from_rinfo(row['rinfo'], feature), axis = 1)

failed_parse.close()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59789 entries, 6132011 to 6785820
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   start_date       59789 non-null  object        
 1   end_date         59789 non-null  object        
 2   status           59789 non-null  object        
 3   req_mem          59789 non-null  int64         
 4   used_mem         59789 non-null  float64       
 5   mem_delta        59789 non-null  float64       
 6   percent_mem      59789 non-null  float64       
 7   dsid             59789 non-null  object        
 8   request_type     59789 non-null  object        
 9   request_id       59789 non-null  float64       
 10  partition_id     45927 non-null  float64       
 11  rinfo            59789 non-null  object        
 12  cpus             56549 non-null  float64       
 13  valid_rinfo      59789 non-null  bool          
 14  dates            12929 non-nul

# Combining redundant columns

The first redundancy we deal with are the columns called "lats" and "lons." 
A "lats" entry, for example, would look like "60 S 80 N." This information should
be split up and entered into the "slat" and "nlat" columns in the same row, by 
setting slat = -60.0 and nlat = 80.0.

In [12]:
def parse_lats_lons(val):
    """Takes a 'lats' or 'lons' value and returns two floats representing the 
    southern/western coordinate and the northern/eastern coordinate.
    Example input: '60 S 80 N'
    Example output: (-60.0, 80.0)
    """
    val = val.replace(',', '')
    substrings = val.split(' ')

    try:
        first_coord = float(substrings[0])
        second_coord = float(substrings[2])
    except:
        print(f"Error expanding lats/lons. Value = {val}")
        return float('nan')

    if substrings[1] == 'W' or substrings[1] == 'S':
        first_coord = -1*first_coord
    if substrings[3] == 'W' or substrings[3] == 'S':
        second_coord = -1*second_coord

    return (first_coord, second_coord)


In [13]:
def update_lat_lon(feature, row):
    """Used to update 'slat', 'nlat', 'wlon', or 'elon' by 
    getting the values from 'lats' or 'lons' in the same row.
    Example input: ('nlat', <row containing 'lats=45 S 50 N'>)
    Example output: 50.0
    """
    if row['lats'] != row['lats']:
        return row[feature]
    else:
        if feature == 'slat':
            return parse_lats_lons(row['lats'])[0]
        elif feature == 'nlat':
            return parse_lats_lons(row['lats'])[1]
        elif feature == 'wlon':
            return parse_lats_lons(row['lons'])[0]
        else:
            return parse_lats_lons(row['lons'])[1]


Compare the df.info() output with the previous df.info(); there should be more non-null slat's, nlat's, etc.

In [14]:
for feature in ['slat', 'nlat', 'wlon', 'elon']:
    df[feature] = df.apply(lambda row: update_lat_lon(feature, row), axis = 1)

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 59789 entries, 6132011 to 6785820
Data columns (total 32 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   start_date       59789 non-null  object        
 1   end_date         59789 non-null  object        
 2   status           59789 non-null  object        
 3   req_mem          59789 non-null  int64         
 4   used_mem         59789 non-null  float64       
 5   mem_delta        59789 non-null  float64       
 6   percent_mem      59789 non-null  float64       
 7   dsid             59789 non-null  object        
 8   request_type     59789 non-null  object        
 9   request_id       59789 non-null  float64       
 10  partition_id     45927 non-null  float64       
 11  rinfo            59789 non-null  object        
 12  cpus             56549 non-null  float64       
 13  valid_rinfo      59789 non-null  bool          
 14  dates            12929 non-nul

Next we deal with the 'dates' column. Here there are three possibilities:

1. The 'dates' field contains two dates, representing a start and end date, 
e.g. "dates=2019-01-01 00:00 2019-12-31 18:00"

2. The 'dates' field says the dates were initial (as opposed to valid),
e.g. "dates=init"

3. The 'dates' field was empty, indicating that the dates were valid.


In [15]:
def parse_dates(feature, dates):
    """Deduces a start date or end date from whatever was in
    the 'dates' column. The entered feature must be either 
    'startdate' or 'enddate'.
    Example input: ('enddate', '2019-01-01 00:00 2019-12-31 18:00')
    Example output: 2019-12-31 18:00 (pandas datetime object)
    """
    dates_split = dates.split(' ')
        
    if len(dates_split) == 4:
        # Typical case: dates=2019-01-01 00:00 2019-12-31 18:00
        if feature == 'startdate':
            date = dates_split[0] + ' ' + dates_split[1]
        else:
            date = dates_split[2] + ' ' + dates_split[3]
    else:
        # Typical cases: either dates=1806-01-01 1900-12-31
        # or dates=197005 201412
        if feature == 'startdate':
            date = dates_split[0]
        else:
            date = dates_split[1]
            
        if '-' not in dates_split[0]:
            date = date[:4] + '-' + date[4:]
                
    return pd.to_datetime(date, errors='coerce')


In [16]:
def update_dates(feature, row):
    """Used to update 'startdate', 'enddate', or 'dates_init' by
    getting the values from the 'dates' column in the same row.
    feature must be either 'startdate', 'enddate', or 'dates_init'.
    """
    dates = row['dates']
    
    if feature == 'dates_init':
        return True if dates == 'init' else False

    if row[feature] == row[feature]:
        return row[feature]
    else:
        if row['dates'] == row['dates'] and row['dates'] != 'init':
            return parse_dates(feature, dates)
        else:
            return pd.NaT


Again, after we combine the info from 'dates', there should be more
non-null 'startdate's and 'enddates' compared to the previous df.info() call.

In [17]:
date_features = ['dates_init', 'startdate', 'enddate']
for feature in date_features:
    df[feature] = df.apply(lambda row: update_dates(feature, row), axis = 1)
    
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59789 entries, 6132011 to 6785820
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   start_date       59789 non-null  object        
 1   end_date         59789 non-null  object        
 2   status           59789 non-null  object        
 3   req_mem          59789 non-null  int64         
 4   used_mem         59789 non-null  float64       
 5   mem_delta        59789 non-null  float64       
 6   percent_mem      59789 non-null  float64       
 7   dsid             59789 non-null  object        
 8   request_type     59789 non-null  object        
 9   request_id       59789 non-null  float64       
 10  partition_id     45927 non-null  float64       
 11  rinfo            59789 non-null  object        
 12  cpus             56549 non-null  float64       
 13  valid_rinfo      59789 non-null  bool          
 14  dates            12929 non-nul

# Final cleaning and saving

At the moment, 'start_date' refers to the date/time that the SLURM job began, whereas 'startdate' refers to the date/time that the requested data starts from. We rename those two to avoid confusion. We do the same for 'end_date' and 'enddate.'

In [18]:
df = df.rename(columns={'start_date': 'job_start',
                        'end_date': 'job_end',
                        'enddate': 'rqst_end',
                        'startdate': 'rqst_start'})


In [19]:
list(df.columns)

['job_start',
 'job_end',
 'status',
 'req_mem',
 'used_mem',
 'mem_delta',
 'percent_mem',
 'dsid',
 'request_type',
 'request_id',
 'partition_id',
 'rinfo',
 'cpus',
 'valid_rinfo',
 'dates',
 'dsnum',
 'elon',
 'rqst_end',
 'format',
 'grid_definition',
 'gui',
 'lats',
 'level',
 'lons',
 'nlat',
 'parameters',
 'product',
 'slat',
 'rqst_start',
 'station',
 'tindex',
 'wlon',
 'dates_init']

In [20]:
training_features = ['request_type', 'job_start',
                     'job_end', 'req_mem', 'used_mem',
                     'mem_delta', 'percent_mem',
                     'dsnum', 'rqst_start', 'rqst_end',
                     'dates_init', 'slat',
                     'nlat', 'wlon', 'elon', 'gui',
                     'parameters', 'level', 'product',
                     'station', 'tindex'] 

training_df = df[training_features]


In [21]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59789 entries, 6132011 to 6785820
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   request_type  59789 non-null  object        
 1   job_start     59789 non-null  object        
 2   job_end       59789 non-null  object        
 3   req_mem       59789 non-null  int64         
 4   used_mem      59789 non-null  float64       
 5   mem_delta     59789 non-null  float64       
 6   percent_mem   59789 non-null  float64       
 7   dsnum         57192 non-null  object        
 8   rqst_start    59748 non-null  datetime64[ns]
 9   rqst_end      59661 non-null  datetime64[ns]
 10  dates_init    59789 non-null  bool          
 11  slat          37966 non-null  float64       
 12  nlat          37966 non-null  float64       
 13  wlon          37987 non-null  float64       
 14  elon          37987 non-null  float64       
 15  gui           59789 non-null

In [22]:
training_path = '/glade/work/jdubeau/job-metrics-training.pkl'
training_df.to_pickle(training_path)
